In [1]:
from __future__ import print_function, division
from builtins import range, input

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd

/home/ubuntu/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/home/ubuntu/anaconda3/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


In [6]:
def get_mnist(limit=None):
    df = pd.read_csv('train.csv')
    data = df.as_matrix()
    np.random.shuffle(data)
    X = data[:, 1:] / 255.0 # data is from 0..255
    Y = data[:, 0]
    if limit is not None:
        X, Y = X[:limit], Y[:limit]
    return X, Y

In [18]:
def plot_latent_space(vae, im_size = 28, n = 20):
    #n:  number of pictures slided each row
    x = np.linspace(-5, 5, n)
    y = np.linspace(-5, 5, n)
    im = np.empty((im_size * n, im_size * n))

    Z = []
    for i, x_ in enumerate(x):
        for j, y_ in enumerate(y):
            z = [x_, y_]
            Z.append(z)
    X_hat = vae.predict_using_Z(Z)

    k = 0
    for i, x_ in enumerate(x):
        for j, y_ in enumerate(y):
            x_hat = X_hat[k]
            k += 1
            x_hat = x_hat.reshape(im_size, im_size)
            im[(n - i - 1) * im_size:(n - i) * im_size, j * im_size:(j + 1) * im_size] = x_hat
    plt.imshow(im)
    plt.show()

In [19]:
tf.reset_default_graph()

In [20]:
class DenseLayer(object):
    def __init__(self, M1, M2, f=tf.nn.relu):
        self.W = tf.Variable(tf.random_normal(shape=(M1, M2)) * 2 / np.sqrt(M1))
        self.b = tf.Variable(np.zeros(M2).astype(np.float32))
        self.f = f

    def forward(self, X):
        with tf.device('/device:GPU:1'):
            return self.f(tf.matmul(X, self.W) + self.b)

In [21]:
class VariationalAutoencoder:
    def __init__(self, D, hidden_layer_sizes):
        self.X = tf.placeholder(tf.float32, shape=(None, D))

        # encoder
        self.encoder_layers = []
        M_in = D
        for M_out in hidden_layer_sizes[:-1]:
            h = DenseLayer(M_in, M_out)
            self.encoder_layers.append(h)
            M_in = M_out

        
        M = hidden_layer_sizes[-1]

        # the encoder's final layer, it has a mean, a standard deviation for each unit
        h_final = DenseLayer(M_in, 2 * M, f=lambda x: x)
        self.encoder_layers.append(h_final)

        # now forward till we get mean and std values
        layer_value = self.X
        for layer in self.encoder_layers:
            layer_value = layer.forward(layer_value)
        self.means = layer_value[:, :M]
        self.stddev = tf.nn.softplus(layer_value[:, M:]) + 1e-6 # add small amount for smoothing
        #log(exp(features) + 1) avoids zeros

        # get a sample of Z
        standard_normal = tf.contrib.distributions.Normal(
        loc=np.zeros(M, dtype=np.float32),
        scale=np.ones(M, dtype=np.float32)
        )
        e = standard_normal.sample(tf.shape(self.means)[0])
        self.Z = e * self.stddev + self.means

        # decoder
        self.decoder_layers = []
        M_in = M
        for M_out in reversed(hidden_layer_sizes[:-1]):
            h = DenseLayer(M_in, M_out)
            self.decoder_layers.append(h)
            M_in = M_out

        # the decoder's final layer could also be sigmoid
        h = DenseLayer(M_in, D, f=lambda x: x)
        self.decoder_layers.append(h)

        # get the reconstructed image
        layer_value = self.Z
        for layer in self.decoder_layers:
            layer_value = layer.forward(layer_value)
        logits = layer_value
        posterior_predictive_logits = logits # save for later

        # get the output
        self.X_hat_distribution = tf.contrib.distributions.Bernoulli(logits=logits)

        # take samples from X_hat
        self.posterior_x_hat = self.X_hat_distribution.sample()
        self.posterior_x_hat_sigmoid = tf.nn.sigmoid(logits) 


        # take random samples from Z
        standard_normal = tf.contrib.distributions.Normal(
          loc=np.zeros(M, dtype=np.float32),
          scale=np.ones(M, dtype=np.float32)
        )

        Z_std = standard_normal.sample(1)
        current_layer_value = Z_std
        for layer in self.decoder_layers:
            current_layer_value = layer.forward(current_layer_value)
        logits = current_layer_value

        x_hat_dist = tf.contrib.distributions.Bernoulli(logits=logits)
        self.x_hat_sampled = x_hat_dist.sample()
        self.x_hat_sigmoid =  tf.nn.sigmoid(logits)

        # generate samples from z space
        # only used for generating visualization
        self.Z_input = tf.placeholder(tf.float32, shape=(None, M))
        layer_value = self.Z_input
        for layer in self.decoder_layers:
            layer_value = layer.forward(layer_value)
        logits = layer_value
        self.x_hat_from_inputs = tf.nn.sigmoid(logits)

        #now cost function
        kl = tf.contrib.distributions.kl_divergence(tf.contrib.distributions.Normal(self.means, self.stddev), standard_normal)
        kl = tf.reduce_sum(kl, axis =1)
        #kl divergence kl(p1|p2) = Sum(p1*log(p1/p2)) + Sum(1-p1 *log(1-p1/1-p2))
        
        lle = tf.reduce_sum(self.X_hat_distribution.log_prob(self.X),1 )
        #gives the log(probability mass function for X, given Zs)
        #elbo term 1
        
        self.elbo = tf.reduce_sum(lle - kl)
        self.train_op = tf.train.AdamOptimizer(learning_rate=0.001).minimize(-self.elbo)

        # session
        self.init_op = tf.global_variables_initializer()
        self.sess = tf.InteractiveSession(config=tf.ConfigProto(allow_soft_placement=True))
        self.sess.run(self.init_op)


    def fit(self, X, epochs=2, batch_sz=64):
        costs = []
        n_batches = len(X) // batch_sz
        print("n_batches:", n_batches)
        for i in range(epochs):
            print("epoch:", i)
            np.random.shuffle(X)
            for j in range(n_batches):
                batch = X[j*batch_sz:(j+1)*batch_sz]
                _, c, = self.sess.run((self.train_op, self.elbo), feed_dict={self.X: batch})
                c /= batch_sz 
                costs.append(-c)
                if j % 100 == 0:
                    print("iter: %d, cost: %.3f" % (j, c))
        plt.plot(costs)
        plt.show()

    def get_means_of_Z(self, X):
        return self.sess.run(
          self.means,
          feed_dict={self.X: X}
        )

    def predict_using_Z(self, Z):
        return self.sess.run(
          self.x_hat_from_inputs,
          feed_dict={self.Z_input: Z}
        )

    def predict_using_X(self, X):
        return self.sess.run(self.posterior_x_hat, feed_dict={self.X: X})

    def predict_using_random_Z(self):
        return self.sess.run((self.x_hat_sampled, self.x_hat_sigmoid))




In [22]:
def mnist():
    X, Y = get_mnist()
    # rectifying to binary digits
    X = (X > 0.5).astype(np.float32)

    vae = VariationalAutoencoder(784, [400, 100, 10, 2])
    vae.fit(X, epochs =10)
    n_samples = 10
    
    # reconstruction
    for j in range(n_samples) :
        i = np.random.choice(len(X))
        x = X[i]
        im = vae.predict_using_X([x]).reshape(28, 28)
        plt.figure()
        plt.subplot(1,2,2)
        plt.imshow(x.reshape(28, 28), cmap='gray')
        plt.title("Original")
        plt.subplot(1,2,2)
        plt.imshow(im, cmap='gray')
        plt.title("Sampled")
        plt.show()

    #sample from Z space
    for j in range(n_samples) :
        _, im_sigmoid = vae.predict_using_random_Z()
        plt.figure()
        plt.imshow(im_sigmoid.reshape(28,28), cmap='gray')
        plt.title("Randomly sampled images from latent space")
        plt.show()
    
    #plot the latent space
    plot_latent_space(vae)
    
    
if __name__ == '__main__':
    mnist()

/home/ubuntu/anaconda3/lib/python3.6/site-packages/tensorflow/python/client/session.py:1711: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


n_batches: 656
epoch: 0
iter: 0, cost: -1386.780
iter: 100, cost: -205.113
iter: 200, cost: -196.397
iter: 300, cost: -187.727
iter: 400, cost: -191.522
iter: 500, cost: -185.339
iter: 600, cost: -180.920
epoch: 1
iter: 0, cost: -186.126
iter: 100, cost: -173.414
iter: 200, cost: -160.988
iter: 300, cost: -160.528
iter: 400, cost: -180.762
iter: 500, cost: -164.403
iter: 600, cost: -166.987
epoch: 2
iter: 0, cost: -170.863
iter: 100, cost: -161.951
iter: 200, cost: -174.498
iter: 300, cost: -163.127
iter: 400, cost: -164.175
iter: 500, cost: -155.652
iter: 600, cost: -161.520
epoch: 3
iter: 0, cost: -171.678
iter: 100, cost: -163.859
iter: 200, cost: -160.691
iter: 300, cost: -152.222
iter: 400, cost: -156.577
iter: 500, cost: -160.467
iter: 600, cost: -157.795


KeyboardInterrupt: 